# Toxic Spans Detection

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

import spacy
import ast
from termcolor import colored

from tqdm import tqdm
import gdown

from utils.processing import get_index_toxic_words, color_toxic_words, f1
from utils.lstm import spacy_tokenizer, get_vocab

sns.set_style('darkgrid')
dev = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
# To plot using LaTeX, sometimes it gives trouble, in that case comment these two lines
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

## Resultados

De los distintos preprocesamientos el que mayor F1 score tiene en `test` es poner los posts con [ ] como completamente tóxicos (**best-model-try2.pt**, *train*=0.6498 , *test*=0.6526), así que usaremos ese.

In [3]:
train = pd.read_csv('../data/tsd_train.csv', converters={'spans':ast.literal_eval})
test = pd.read_csv('../data/tsd_trial.csv', converters={'spans':ast.literal_eval})

In [ ]:
# Nuestros embeddings
vocab = get_vocab(train)

In [5]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, stacked_layers, dropout_p, weight, hidden_dim, vocab_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim         # Dimension del estado oculta en cada direccion de la LSTM
        self.stacked_layers = stacked_layers # Cuantas capas en la LSTM
        
        self.word_embeddings = nn.Embedding.from_pretrained(weight)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers=stacked_layers,
                            dropout=dropout_p,
                            bidirectional=True)

        # Linear layers
        self.fc1 = nn.Linear(hidden_dim*2, 1) # 2 veces el tamaño de hidden_dim por ser bidireccional

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        output, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        x = torch.sigmoid(self.fc1(output.view(len(sentence), -1)))
        return x

In [6]:
def prepare_sequence(seq):
    idxs = vocab.lookup_indices(seq)      # Si no está lo pone como 0
    return torch.tensor(idxs, dtype=torch.long, device=dev)

def prepare_sequence_tags(seq):
    tag_to_ix = {"non_toxic": 0, "toxic": 1} 
    idxs = [tag_to_ix[s] for s in seq]
    return torch.tensor(idxs, dtype=torch.long, device=dev)

def tagger_LSTM(text, threshold=0.5):
    """
    Hace el tagging con el modelo que entrenamos.
    """
    ix_to_tag = {0: 'non_toxic', 1: 'toxic'}
    words = spacy_tokenizer(text.lower()) # Parece funcionar mejor
    
    with torch.no_grad():
        inputs = prepare_sequence(words)
        tag_scores = model(inputs)
        
        tags = [1 if x > threshold else 0 for x in tag_scores]
        tagged_sentence = list(zip(words, tags))

    return tagged_sentence

El archivo con el modelo está algo pesado para GitHub, así que lo pondremos en un Drive para descargarlo.

In [ ]:
url = 'https://drive.google.com/uc?id=1KO-QXUBfwzjauWLhiVi9StD3y0GtiBbj'
output = 'models/best-model.pt'

gdown.download(url, output, quiet=False) 

In [8]:
model = torch.load(output)
model.to(torch.device(dev))

LSTMTagger(
  (word_embeddings): Embedding(19611, 200)
  (lstm): LSTM(200, 600, num_layers=6, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=1200, out_features=1, bias=True)
)

In [9]:
indices_test = []
for i, (gold_index, text) in enumerate(zip(test['spans'],test['text'])):
    tagged_sentence = tagger_LSTM(text)   
    prediction_index = get_index_toxic_words(text.lower(), tagged_sentence)
    indices_test.append(prediction_index)
    
    if i < 5:
        print(str(i) + colored(' Pred: ', color='cyan', attrs=['bold']) + 
              color_toxic_words(prediction_index, text))
        print(str(i) + colored(' Gold: ', color='yellow', attrs=['bold']) + 
              color_toxic_words(gold_index, text) + '\n'*2)

0 Pred: Because he's a moron and a bigot. It's not any more complicated than that.
0 Gold: Because he's a moron and a bigot. It's not any more complicated than that.


1 Pred: How about we stop protecting idiots and let nature add some bleach to the gene pool. We can always submit their names for the Darwin awards.
1 Gold: How about we stop protecting idiots and let nature add some bleach to the gene pool. We can always submit their names for the Darwin awards.


2 Pred: If people  were  smart, they would  Boycott this  inept  airline,  but   they  are  not  smart,  so   rogue  businesses  like  this  one,   still thrive   taking the idiots  for  a ride...
2 Gold: If people  were  smart, they would  Boycott this  inept  airline,  but   they  are  not  smart,  so   rogue  businesses  like  this  one,   still thrive   taking the idiots  for  a ride...


3 Pred: Trump Claimed that Russia will never invade the Ukraine, when Russia already has - how stupid can people be?
3 Gold: Trump Claim

In [10]:
score_test = [f1(pred, gold) for pred,gold in zip(indices_test, test['spans'])]
print('F1 in test: {:.6f}'.format(np.mean(score_test)))

F1 in test: 0.648722


## Predicción en el dataset de *evaluation*

In [11]:
evaluation = pd.read_csv('../data/tsd_test.csv')

In [12]:
indices_evaluation = []
for i,text in enumerate(evaluation['text']):
    tagged_sentence = tagger_LSTM(text)   
    prediction_index = get_index_toxic_words(text.lower(), tagged_sentence)
    indices_evaluation.append(prediction_index)
    
    if i < 5:
        print(str(i) + colored(' Pred: ', color='cyan', attrs=['bold']) + 
              color_toxic_words(prediction_index, text) + '\n') 

0 Pred: That's right. They are not normal. And I am starting from the premise that they are ABNORMAL. Proceed wth the typical racist, bigot, sexist rubbish.
Thanks!

1 Pred: "Watch people die from taking away their healthcare"

DING DING DING!  Winner of stupid post of the day award!

2 Pred: tens years ago i contacted the PDR and suggested that the time might be good to work with alaska on building a gas line.. alaska rejected them without even consideration despite china being flush with cash and hungry for gas.. and set up another infamous boondoggle.. the transcanada-exxon rip off that we are still paying for and have yet to receive anything of value.. hundreds of millions of dollars on studies.. and buyouts.. i hope china says f you alaska.. you are nothing but ignorant people..

3 Pred: The parallels between the ANC and the Sicilian Mafia are glaring. The ANC has always been run by a few "families" who treat the state as 'turf' ; as just one big piggy bank for their self-enrichme

In [13]:
evaluation['spans'] = indices_evaluation
evaluation = evaluation[['spans', 'text']]
evaluation.head()

,spans,text
0,"[118, 119, 120, 121, 122, 123, 124, 125, 126, ...",That's right. They are not normal. And I am st...
1,"[81, 82, 83, 84, 85, 86]","""Watch people die from taking away their healt..."
2,"[483, 484, 485, 486, 487, 488, 489, 490]",tens years ago i contacted the PDR and suggest...
3,"[413, 414, 415, 416, 417, 418, 419, 420]",The parallels between the ANC and the Sicilian...
4,"[150, 151, 152, 153, 154, 155, 156, 271, 272, ...",Intel Community: ‘How can we work for a Presid...


Para la evaluación se debe subir un zip con un archivo txt de la siguiente manera (al final subir el archivo `spans-pred.zip` que se produce):

In [14]:
predictions = evaluation['spans'].tolist()
ids = evaluation.index.tolist()

with open("spans-pred.txt", "w") as out:
    for uid, text_scores in zip(ids, predictions):
        out.write(f"{str(uid)}\t{str(text_scores)}\n")
        
# Zip the predictions
! zip -r spans-pred.zip ./spans-pred.* 
! rm spans-pred.txt
! mv spans-pred.zip ../spans-pred.zip

  adding: spans-pred.txt (deflated 84%)
